[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/modelagem_1.ipynb)

# Modelagem e Gestão de Processos


**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**

Tema da aula - Modelagem de Processos com a biblioteca SimPy

# SimPy

A biblioteca SimPy apresenta um framework útil para simular o funcionamento de processos em Python. 

O usuário ganha grande liberdade para modelar processos específicos de sua organização. 

Além disso, pode integrar ao modelo as inúmeras bibliotecas existentes em python para Estatística, Ciência de Dados, Aprendeizado de Máquina, Otimização, etc.

## Importando o SimPy

Se estiver trabalhando localmente, uma vez instalado o Python, você deverá instalar a biblioteca SimPy antes de importá-la. 

- Isso pode ser feito com o comando: **pip install simpy**

No caso do Googlo Colab: **!pip install simpy**

Uma vez instalada, podemos importar a biblioteca normalmente simplesmente com: **import simpy**. 

In [1]:
#!pip install simpy

import simpy
import numpy as np
import random

In [2]:
# Seu computador não gera números verdadeiramente aleatórios, mas sim, pseudoaleatórios.
# Vamos "travar" a semente de geração desses números. Assim, resultados poderão ser reproduzidos no futuro.
random.seed(10)

# Um exemplo simples: Pomodoro

Com o intuito de aumentar sua produtividade, um estudante/trabalhador resolve seguir a recomendação de uma colega sobre a metodologia Pomodoro.

- Ele fará alternadamente intervalos de trabalho e pausa (descanso, café, água, banheiro) durante o dia.

    - O tempo até a próxima parada para descanso (tempo de trabalho) segue uma distribuição uniforme que varia entre 24 e 26 minutos.
    - As pausas seguem uma exponencial de média 5 minutos.

Vamos modelar o funcionamento desse processo por 8 horas de trabalho e imprimir em nossa tela sempre que um evento (pausa ou trabalho) for iniciado.

## Criando um ambiente de simulação


O primeiro passo será criar um ambiente (environment) no SimPy. 

In [3]:
# Criando uma instância de ambiente do simpy e armazenando em env

env = simpy.Environment()

Nosso processo possui duas atividades (subprocessos): **trabalhar** e **descansar**. 

A próxima etapa é criar um **generator** (tipo especial de função) para simular o funcionamento dessas atividades. 

### Modelando o Processo

In [4]:
# Criaremos um gerador de nome pomodoro 

def pomodoro ():
    pass 

Por enquanto, não passamos nada para nosso gerador. 

Os parâmetros que ele irá receber devem incluir o ambiente criado no simpy. 

Vamos também passar como parâmetro o nome do colaborador.

In [5]:
def Pomodoro (environment, nome):
    pass 

Agora, podemos trabalhar nas atividades que farão parte do nosso processo: **trabalhar** e **descansar**.

Para isso, usaremos o **yield** e os métodos **now** e **timeout** do simpy.

- O **yield** é usado em uma função geradora (em detrimento do return numa função normal). No caso do yield, guardamos um iterador (no nosso caso será o tempo de início de cada evento) que poderá ser retomado no futuro e usado em outro chamado da função geradora. 

- O método **now** do SimPy retorna o tempo de simulação no nosso ambiente enquanto o **timeout** avisa à simulação para percorrer um tempo.

In [6]:
def pomodoro (environment, nome):
    
    # enquanto a simulação estiver ocorrendo
    while True:
        
        # imprima na tela o tempo de início do evento e depois percorra um tempo random.uniform(24, 26)
        print (f"{nome} inicia trabalho no tempo {environment.now}")
        yield environment.timeout(random.uniform(24,26))

        # imprima na tela o tempo de início do evento e depois percorra um tempo random.expovariate(1/média)
        print (f"{nome} inicia descanso no tempo {environment.now}")
        yield environment.timeout(random.expovariate(1/5))

Nosso gerador "pomodoro" ainda não é visto como um processo do nosso environment no SimPy. 

O próximo passo é instanciar nosso processo usando o simpy.Environment.process. 

No nosso caso, fica simplesmente **env.process**.

In [7]:
# Instância do processo
env.process(pomodoro(env, "Will_Smith"))

<Process(pomodoro) object at 0x11c29bf1af0>

Agora, vamos rodar o nosso processo até o minuto 480, ou seja, simularemos 8h de trabalho.

In [8]:
env.run (until = 480)

Will_Smith inicia trabalho no tempo 0
Will_Smith inicia descanso no tempo 25.14280518937983
Will_Smith inicia trabalho no tempo 27.943664130161597
Will_Smith inicia descanso no tempo 53.099846732430535
Will_Smith inicia trabalho no tempo 54.253824449673225
Will_Smith inicia descanso no tempo 79.88046695238786
Will_Smith inicia trabalho no tempo 88.55515723500187
Will_Smith inicia descanso no tempo 113.86210230280423
Will_Smith inicia trabalho no tempo 114.7352358331193
Will_Smith inicia descanso no tempo 139.77657455239915
Will_Smith inicia trabalho no tempo 141.76236914065487
Will_Smith inicia descanso no tempo 166.26236249402768
Will_Smith inicia trabalho no tempo 181.53096096998934
Will_Smith inicia descanso no tempo 207.52407495506824
Will_Smith inicia trabalho no tempo 207.75197258188612
Will_Smith inicia descanso no tempo 233.4722946564587
Will_Smith inicia trabalho no tempo 238.09379086634283
Will_Smith inicia descanso no tempo 262.8570028381811
Will_Smith inicia trabalho no tem

# Modelando um Processo com Várias Atividades

Agora vamos modelar um processo um pouco mais complexo. Teremos chegadas de clientes em um serviço composto por três atividades: A, B e C.

-  Um processo de chegadas inicializa a cada chegada de um cliente. O tempo entre chegadas segue uma distribuição Exponencial com média de 5 minutos. 
     
$$\frac{1}{\lambda}=5 \rightarrow \lambda = 0,2$$ 

- Cada cliente segue o seguinte percurso:
    - Serviço A (tempo de processamento 3 min)
    - Serviço B (tempo de processamento de 8 min)
    - Serviço C (tempo de processamento segue uma distribuição exponencial com média 5min)

Para a realização dos serviços, precisamos de recursos. Por exemplo, atendentes, caixas, etc. 

Chamaremos os recursos de:

- colaborador_A
- colaborador_B
- colaborador_C

Para isso, utilizaremos **recursos** do SimPy, criados com **resource_name = env.Resource (env, capacity)**.

Um cliente deve esperar até que o recurso esteja disponível. Para modelar isso, usaremos o **resource_name.request()**

### Função Geradora

In [9]:
# Função geradora receberá, além do ambiente e do nome do cliente, os recursos.

def salao (environment, nome, colaborador_A, colaborador_B, colaborador_C):
    
    # imprimimos na tela quando um cliente chega no estabelecimento       
    print (f"{nome} chega no estabelecimento em {env.now}")
    
    # Serviço A
    # Ao chegar, o cliente deve esperar o recurso do primeiro serviso

    with colaborador_A.request() as req_A:
        yield req_A # o cliente deve esperar um recurso do tipo colaborador_A
        #print (f"{nome} inicia A em {env.now}") # imprima na tela o tempo de início
        yield environment.timeout(3) # O serviço demora exatamente 3 minutos.
        #print (f"{nome} finaliza A em {env.now}")

    
    # Serviço B
    with colaborador_B.request() as req_B:
        yield req_B
        #print (f"{nome} inicia B em {env.now}")
        yield environment.timeout(8) # O serviço B demora exatamente 8 minutos.
        #print (f"{nome} finaliza B em {env.now}")
    
    # Serviço C
    with colaborador_C.request() as req_C:
        yield req_C
        #print (f"{nome} inicia C em {env.now}")
        yield environment.timeout(random.expovariate(1/5)) # O tempo de serviço em C segue uma exponencial de média 5.
    
    print (f"{nome} sai do estabelecimento em {environment.now}")

    

### Definindo o ambiente

In [10]:
env_salao = simpy.Environment()

### Instanciando os tipos de recurso em nosso ambiente e suas capacidades

In [11]:
colab_A = simpy.Resource(env_salao, capacity=1)
colab_B = simpy.Resource(env_salao, capacity=1)
colab_C = simpy.Resource(env_salao, capacity=1)

## Processo de Chegadas


Vamos considerar que o nosso processo de chegadas é um processo de Poisson, com $\lambda = 0,2$ clientes/minuto;

Assim, o tempo entre chegadas segue uma distribuição exponencial de média $1/\lambda = 5$ minutos.

Vamos criar nosso processo de chegadas:

In [12]:
def chegadas (environment):
    i=1 # número que será usado na contagem e nomeação dos clientes    
    # Enquanto a simulação ocorrer
    while True:
        # gere um número de nossa distribuição 
        yield environment.timeout(random.expovariate(1/5))
        # Um cliente entra no processo! Seu nome será "Cliente i"
        environment.process(salao (environment, 'Cliente %d' % i, colab_A, colab_B, colab_C))        
        i+=1 # i = i+1

Pronto, agora basta rodar nossa simulação.

In [13]:
env_salao.process(chegadas(env_salao)) # Instanciando o processo com a função de chegadas.

env_salao.run(until=480)

Cliente 1 chega no estabelecimento em 480
Cliente 2 chega no estabelecimento em 480
Cliente 3 chega no estabelecimento em 480
Cliente 4 chega no estabelecimento em 480
Cliente 5 chega no estabelecimento em 480
Cliente 6 chega no estabelecimento em 480
Cliente 7 chega no estabelecimento em 480
Cliente 8 chega no estabelecimento em 480
Cliente 9 chega no estabelecimento em 480
Cliente 10 chega no estabelecimento em 480
Cliente 1 sai do estabelecimento em 31.715214464342633
Cliente 11 chega no estabelecimento em 480
Cliente 12 chega no estabelecimento em 480
Cliente 2 sai do estabelecimento em 40.23348668860151
Cliente 13 chega no estabelecimento em 480
Cliente 14 chega no estabelecimento em 480
Cliente 15 chega no estabelecimento em 480
Cliente 16 chega no estabelecimento em 480
Cliente 3 sai do estabelecimento em 49.89293214123484
Cliente 4 sai do estabelecimento em 50.492008460561074
Cliente 17 chega no estabelecimento em 480
Cliente 5 sai do estabelecimento em 54.93152298837021
Client

Ótimo, já modelamos 2 processos!
 
Porém, não estamos guardando informações para calcular estatísticas que nos interessem.

In [14]:
#criando listas vazias
tempo_gasto_C = []
tempo_esperando_C =[]

def salao (environment, nome, colaborador_A, colaborador_B, colaborador_C):
    #print (f"{nome} chega no estabelecimento em {env.now}")
    # Serviço A
    with colaborador_A.request() as req_A:
        yield req_A # o cliente deve esperar um recurso do tipo colaborador_A
        #print (f"{nome} inicia A em {env.now}") # imprima na tela o tempo de início
        yield environment.timeout(3) # O serviço demora exatamente 3 minutos.
        #print (f"{nome} finaliza A em {env.now}")
    
    # Serviço B
    with colaborador_B.request() as req_B:
        yield req_B
        #print (f"{nome} inicia B em {env.now}")
        yield environment.timeout(8) # O serviço B demora exatamente 8 minutos.
        #print (f"{nome} finaliza B em {env.now}")

        acaba_B = environment.now

    # Serviço C
    with colaborador_C.request() as req_C:
        yield req_C
        #print (f"{nome} inicia C em {env.now}")
        
        p_inicio = environment.now
        yield environment.timeout(random.expovariate(1/5)) # O tempo de serviço em C segue uma exponencial de média 5.
        p_fim = environment.now

        tempo_gasto_C.append(p_fim - p_inicio)
        tempo_esperando_C.append(p_inicio - acaba_B)
        
    #print (f"{nome} sai do estabelecimento em {env.now}")
    print (f"{nome} esperou {p_inicio - acaba_B} na fila da atividade C")
    print (f"{nome} gastou {p_fim - p_inicio} na atividade C")


def chegadas (env):
    i=1 # número que será usado na contagem e nomeação dos clientes
    # Enquanto a simulação ocorrer
    while True:
        # gere um número de nossa distribuição 
        yield env.timeout(random.expovariate(1/5))
        # Um cliente entra no processo! Seu nome será "Cliente i"
        env.process(salao (env, 'Cliente %d' % i, colab_A, colab_B, colab_C))
        i+=1 # i = i+1


In [15]:
env_salao_alterado = simpy.Environment()
colab_A = simpy.Resource(env_salao_alterado, capacity=1)
colab_B = simpy.Resource(env_salao_alterado, capacity=1)
colab_C = simpy.Resource(env_salao_alterado, capacity=1)
env_salao_alterado.process(chegadas(env_salao_alterado))
env_salao_alterado.run(until = 6000)

Cliente 1 esperou 0.0 na fila da atividade C
Cliente 1 gastou 0.08226210443761772 na atividade C
Cliente 2 esperou 0.0 na fila da atividade C
Cliente 2 gastou 1.943756968837853 na atividade C
Cliente 3 esperou 0.0 na fila da atividade C
Cliente 3 gastou 4.711971012669444 na atividade C
Cliente 4 esperou 0.0 na fila da atividade C
Cliente 4 gastou 13.723758764044732 na atividade C
Cliente 5 esperou 5.723758764044732 na fila da atividade C
Cliente 5 gastou 1.0642784246346366 na atividade C
Cliente 6 esperou 0.0 na fila da atividade C
Cliente 6 gastou 4.241927529946544 na atividade C
Cliente 7 esperou 0.0 na fila da atividade C
Cliente 7 gastou 2.8929594617060417 na atividade C
Cliente 8 esperou 0.0 na fila da atividade C
Cliente 8 gastou 10.185432999406473 na atividade C
Cliente 9 esperou 2.1854329994064727 na fila da atividade C
Cliente 9 gastou 0.08089139705820969 na atividade C
Cliente 10 esperou 0.0 na fila da atividade C
Cliente 10 gastou 4.26940996401423 na atividade C
Cliente 11 e

In [16]:
print(f"Em média, um cliente o serviço C dura {np.mean(tempo_gasto_C)} minutos")
print(f"Em média, um cliente espera {np.mean(tempo_esperando_C)} minutos pelo serviço C")

Em média, um cliente o serviço C dura 4.947990593834371 minutos
Em média, um cliente espera 2.700315916221725 minutos pelo serviço C


---


# Exercício 1


**Hospital Municipal**

O administrador de um hospital concluiu que os casos de emergência chegam, em sua maioria, de forma aleatória (um processo de entrada de Poisson), de modo que os tempos entre atendimentos possuem uma distribuição exponencial.

Ele também concluiu que o tempo gasto por um médico tratando os casos segue, aproximadamente, uma distribuição exponencial. 

Ele optou pelo modelo M/M/1 para um estudo preliminar desse sistema de filas.

Projetando para o ano que vem os dados disponíveis para o turno do início da noite, ele estima que os pacientes chegarão em uma taxa média de 1 a cada 1/2 hora. O médico responsável demora em média de 20 minutos para atender cada paciente. 

Portanto, com uma hora sendo a unidade de tempo, temos:

- $\lambda = 2$ clientes por hora.

- $\mu = 3$ clientes por hora.

Modele o processo e calcule o tempo médio que um cliente passa em fila.

O resultado está prócimo do esperado pela fórmula? 

$$\frac{\lambda}{\mu(\mu - \lambda)} = 2/3 = 0,6667$$


In [ ]:
random.seed(20)

espera = [] # lista inicialmente vazia
def hospital (environment, paciente, medico):
    
    # 1 - na célula abaixo, guarde o momento em que um paciente chega no hospital  
    chegou = 
    
    with medico.request() as req_medico:
        
        # 2 - Ao chegar, o cliente requisita um médico! 
        yield 
        
        # 3 - Quanto o cliente esperou? Insira na lista espera abaixo
        espera.append()
        
        # 4 - complete o yield abaixo atribuindo a distribuição mencionada no enunciado como tempo de processamento da consulta 
        yield 
        


def chegadas (environment):
    i=1 
    while True:
        # 5 - complete o yield abaixo. Quanto tempo se passa até o próximo paciente chegar?
        yield 
        
        environment.process(hospital (environment, 'Paciente %d' % i, medico))
        i+=1


Para testar o modelo, rode a célula abaixo:

In [ ]:
env_MM1 = simpy.Environment() # cria o ambiente 
medico = simpy.Resource(env_MM1, capacity=1) # adiciona o recurso "medico" ao ambiente, com capacidade = 1.
env_MM1.process(chegadas(env_MM1)) # o processamento ocorrerá de acordo com a função chegadas.
env_MM1.run(until = 100000) # o modelo será simulado por 100000 horas.


np.mean(espera)

# Exercício 2

### Modelo M/M/3

Digamos que agora o hospital possua 3 médicos a disposição em qualquer momento do dia. Além disso, temos as seguintes taxas ao longo do dia:

- Taxa de chegadas: $\lambda = 8$ clientes/hora
- Taxa de atendimento de cada médico: $\mu = 3$ clientes/hora

Simule 1 dia de funcionamento desse hospital (como as taxas já estão em clientes/hora, use "until = 24"). 

Qual tamanho tempo médio de espera na fila que vc encontrou?

In [ ]:
# Trabalhe aqui
espera_exercicio_Q2 = []
def hospital_exercicio (environment, nome, atendente):
    chegou = environment.now
    with medico.request() as req_medico:
        yield req_medico
        espera_exercicio_Q2.append(environment.now - chegou)
        # EXERCÍCIO: Complete o yield abaixo com a função environment.timeout 
        yield 

def chegadas_exercicio (environment):
    i=1 
    while True:
        # EXERCÍCIO: Complete o yield abaixo com a função environment.timeout
        yield       
        environment.process(hospital_exercicio (environment, 'Paciente %d' % i, medico))
        i+=1

ambiente_Q2 = simpy.Environment()
# EXERCÍCIO: Atribua abaixo uma capacidade para o seu recurso
medico = simpy.Resource(ambiente_Q2, capacity = )
ambiente_Q2.process(chegadas_exercicio(ambiente_Q2))
# EXERCÍCIO: Indique o tempo de simulação abaixo
ambiente_Q2.run(until = )

import numpy as np
print (f"O tempo de espera médio observado em 24h de simulação foi: {np.mean(espera_exercicio_Q2).round(2)} horas")

## Questão 3

E se fossem apenas 2 médicos para as mesmas taxas do Exercício 2?  

Dica: lembre de mudar o nome da variável que você atribuirá o ambiente com o método **simpy.Environment()**

Trabalhe na célula abaixo